In [1]:
%env KERAS_BACKEND = tensorflow
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.optimizers import SGD

from keras.datasets import mnist
from keras.utils import np_utils

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [3]:
print("總共有%d訓練資料，每筆資料尺寸為 %d x %d" %x_train.shape)
print("總共有%d訓練資料，每筆資料尺寸為 %d x %d" %x_test.shape)

總共有60000訓練資料，每筆資料尺寸為 28 x 28
總共有10000訓練資料，每筆資料尺寸為 28 x 28


In [4]:
#因此用reshape調校一下。
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

In [6]:
print(x_train.shape,x_test.shape)

(60000, 784) (10000, 784)


In [22]:
#並將label轉換成one-hot encoding的形式
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

In [23]:
from keras.models import Model

In [24]:
from keras.layers import Input

In [25]:
f_1 = Dense(500,activation='sigmoid')
f_2 = Dense(500,activation='sigmoid')
f_3 = Dense(10,activation='softmax')

In [26]:
print(f_1)

In [27]:
#基本上只有第一個變數需獨立定義，用input函數來定義
x = Input(shape=(784,))

In [28]:
print(x)

Tensor("input_2:0", shape=(?, 784), dtype=float32)


In [29]:
h_1 = f_1(x)
h_2 = f_2(h_1)
y = f_3(h_2)

In [30]:
print(h_1)
print(h_2)
print(y)

Tensor("dense_4/Sigmoid:0", shape=(?, 500), dtype=float32)
Tensor("dense_5/Sigmoid:0", shape=(?, 500), dtype=float32)
Tensor("dense_6/Softmax:0", shape=(?, 10), dtype=float32)


In [31]:
#將輸入、輸出包裝起來，Model(input變數,output變數)
model = Model(x,y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_5 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5010      
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


In [32]:
#組裝
model.compile(loss='mse',optimizer=SGD(lr=0.1),metrics=['accuracy'])

In [33]:
#訓練神經網路，訓練幾筆資料(batch_size)，和共訓練幾次(epochs)
model.fit(x_train, y_train, batch_size=100, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 8s 130us/step - loss: 0.0834 - acc: 0.3562
Epoch 2/5
60000/60000 [==============================] - 8s 136us/step - loss: 0.0635 - acc: 0.5924
Epoch 3/5
60000/60000 [==============================] - 7s 110us/step - loss: 0.0461 - acc: 0.7597
Epoch 4/5
60000/60000 [==============================] - 7s 110us/step - loss: 0.0338 - acc: 0.8409
Epoch 5/5
60000/60000 [==============================] - 6s 105us/step - loss: 0.0261 - acc: 0.8725


In [34]:
#讀取以訓練好的神經網路權重
model.load_weights('handwriting_model_weights.h5')

In [35]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 1s 63us/step


[0.010728656833944842, 0.9311]

In [36]:
#非線性堆疊模型
from keras.layers import concatenate,add

In [39]:
f_4 = Dense(500,activation='relu')
z = f_4(h_1)

#new f_3
f_3 = Dense(10,activation='softmax')
#此段錯誤，y=f_3(h_2,2)，因為funtional API只吃一個量
u = concatenate([h_2,z])
y = f_3(u)

In [40]:
print(u)
print(y)

Tensor("concatenate_1/concat:0", shape=(?, 1000), dtype=float32)
Tensor("dense_8/Softmax:0", shape=(?, 10), dtype=float32)


In [41]:
model = Model(x,y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 500)          392500      input_2[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 500)          250500      dense_4[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 500)          250500      dense_4[0][0]                    
__________________________________________________________________________________________________
concatenat

In [42]:
#引入lambda函數，將python上的function包裝成keras上的layer
from keras.layers.core import Lambda
from keras import backend as K

In [43]:
def average_function(input):
    return K.mean(inputs,axis=1)

In [44]:
#函式不接受numpy array的類型輸入，因此average_function(np.array([1,3]))不成立
average_layer = Lambda(average_function)
print(average_layer)

In [46]:
#接受四維向量，輸出一維向量
inputSize = 4

x = Input(shape=(inputSize,))
y = average_layer(x)
average_model = Model(x,y)

NameError: name 'inputs' is not defined